In [ ]:
import pandas as pd
import numpy as np

# Data

In [ ]:
DATA_PATH = 'DATA_PATH'
df = pd.read_csv(DATA_PATH + 'data.csv')

In [ ]:
df.target = df.target.str.replace('\xad', '')
df.target = df.target.str.replace('\u00ad', '')
df.target = df.target.str.replace('\t', '')
df.target = df.target.str.replace('\N{SOFT HYPHEN}', '')

In [ ]:
df.head()

,Unnamed: 0,target,source
0,0,Та пыгрисит маим вармаль э̄рнэ поратэт ат верм...,Те мальчики не выполнят задание в назначенный ...
1,1,"Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс.","Бегая к берегу озера пришла, к воде подбежала."
2,2,Вит са̄мыл сунсым о̄нтыс,Вода прибывала на глазах
3,3,"Атаявев, акваг лылынг тагл ворн та тотавев.","Обнюхивает нас, живыми на кладбище уносит."
4,4,"Ман ты пӣлтал, веськат хумиюв нэтхуньт ат ёр...",Мы никогда не забудем этого честного человека.


In [ ]:
len(df)

81146

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_test_split = train_test_split(df, test_size=0.1, random_state=121)

In [ ]:
df_test, df_validation = train_test_split(df_test_split, test_size=0.8, random_state=121)

In [ ]:
len(df_test)

1623

# Evaluation

In [ ]:
import os
from transformers import AutoModelForSeq2SeqLM, NllbTokenizer


MODEL_SAVE_PATH = 'MODEL_SAVE_PATH'
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_SAVE_PATH)
tokenizer = NllbTokenizer.from_pretrained(MODEL_SAVE_PATH)


In [ ]:
model.cuda();

In [ ]:
def translate(
    text, src_lang='rus_Cyrl', tgt_lang='eng_Latn',
    a=32, b=3, max_input_length=1024, num_beams=4, **kwargs
):
    """Turn a text or a list of texts into a list of translations"""
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    inputs = tokenizer(
        text, return_tensors='pt', padding=True, truncation=True,
        max_length=max_input_length
    )
    model.eval() # turn off training mode
    result = model.generate(
        **inputs.to(model.device),
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        num_beams=num_beams, **kwargs
    )
    return tokenizer.batch_decode(result, skip_special_tokens=True)

In [ ]:
# Example usage:
t = 'А̄лы ма̄т о̄лнэ то̄ва ӯйхулт вит та̄нти палтыланыл ӯньщим о̄луӈкв ханьщувласт,  таи ма̄гсыл хо̄са вит тал о̄луӈкв вēрмегы.'
print(translate(t, 'mns_Cyrl', 'rus_Cyrl'))
# ['Обитатели южных районов научились накапливать в организме запасы влаги, по этому по долгу могут обходиться без воды.']

['Обитатели южных районов научились накапливать в организме запасы влаги, по этому по долгу могут обходиться без воды.']


In [ ]:
# Example usage:
t = 'Его много народу знает, во многих странах известен.'
print(translate(t, 'rus_Cyrl', 'mns_Cyrl'))
# ['Тав са̄в хо̄тпан ва̄ве, са̄в ма̄т ва̄ве.']

['Тав са̄в хо̄тпан ва̄ве, са̄в ма̄т ва̄ве.']


In [ ]:
def batched_translate(texts, batch_size=16, **kwargs):
    """Translate texts in batches of similar length"""
    idxs, texts2 = zip(*sorted(enumerate(texts), key=lambda p: len(p[1]), reverse=True))
    results = []
    for i in trange(0, len(texts2), batch_size):
        results.extend(translate(texts2[i: i+batch_size], **kwargs))
    return [p for i, p in sorted(zip(idxs, results))]

In [ ]:
from tqdm.auto import tqdm

In [ ]:
df_test['source_translated'] = [translate(t, 'mns_Cyrl', 'rus_Cyrl')[0] for t in tqdm(df_test.target)]
df_test['target_translated'] = [translate(t, 'rus_Cyrl', 'mns_Cyrl')[0] for t in tqdm(df_test.source)]

  0%|          | 0/1623 [00:00<?, ?it/s]

  0%|          | 0/1623 [00:00<?, ?it/s]

In [ ]:
import sacrebleu
bleu_calc = sacrebleu.BLEU(max_ngram_order=4)
chrf_calc = sacrebleu.CHRF(word_order=2)  # this metric is called ChrF++   word_order=2

print(bleu_calc.corpus_score(df_test['source_translated'].tolist(), [df_test['source'].tolist()]))
print(chrf_calc.corpus_score(df_test['source_translated'].tolist(), [df_test['source'].tolist()]))
print(bleu_calc.corpus_score(df_test['target_translated'].tolist(), [df_test['target'].tolist()]))
print(chrf_calc.corpus_score(df_test['target_translated'].tolist(), [df_test['target'].tolist()]))

BLEU = 23.11 55.9/27.5/16.9/11.1 (BP = 0.998 ratio = 0.998 hyp_len = 16529 ref_len = 16562)
chrF2++ = 47.96
BLEU = 24.49 55.6/29.7/19.3/13.3 (BP = 0.961 ratio = 0.961 hyp_len = 15377 ref_len = 15995)
chrF2++ = 51.57
